# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 3790, done.
remote: Counting objects: 100% (1263/1263), done.
remote: Compressing objects: 100% (479/479), done.
remote: Total 3790 (delta 694), reused 1121 (delta 606), pack-reused 2527 (from 1)
Receiving objects: 100% (3790/3790), 164.84 MiB | 32.45 MiB/s, done.
Resolving deltas: 100% (2213/2213), done.
Updating files: 100% (364/364), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 24.0 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/11]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1252:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationI

In [ ]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithoutKFCV/SLIM/OptimizingMAP'

np.random.seed(42)

## Import the repository

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'SLIMElasticNet',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_SLIMElasticNet.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/history_{config["model"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [12]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

def objective_function_SLIMElasticNet(optuna_trial):
    
    recommender_instance = SLIMElasticNetRecommender(URM_train)
    
    full_hyperp = {
                   "topK": optuna_trial.suggest_int("topK", 500, 1500),
                   "l1_ratio": optuna_trial.suggest_float("l1_ratio", 0.01, 1.0, log=True),
                    "alpha": optuna_trial.suggest_float("alpha", 1e-4, 1e-3, log=True),
                    "positive_only": optuna_trial.suggest_categorical("positive_only", [True, False]),
                  }        
    
    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_SLIMElasticNet, n_trials=5)

[I 2024-12-02 18:28:08,992] Using an existing study with name 'hyperparameters_tuning_SLIMElasticNet' instead of creating a new one.


SLIMElasticNetRecommender: Processed 2575 ( 6.8%) in 5.00 min. Items per second: 8.58
SLIMElasticNetRecommender: Processed 5224 (13.7%) in 10.00 min. Items per second: 8.70
SLIMElasticNetRecommender: Processed 7963 (20.9%) in 15.00 min. Items per second: 8.84
SLIMElasticNetRecommender: Processed 11291 (29.6%) in 20.00 min. Items per second: 9.41
SLIMElasticNetRecommender: Processed 14622 (38.4%) in 25.01 min. Items per second: 9.75
SLIMElasticNetRecommender: Processed 17780 (46.6%) in 30.01 min. Items per second: 9.88
SLIMElasticNetRecommender: Processed 20896 (54.8%) in 35.01 min. Items per second: 9.95
SLIMElasticNetRecommender: Processed 23841 (62.5%) in 40.01 min. Items per second: 9.93
SLIMElasticNetRecommender: Processed 26972 (70.8%) in 45.01 min. Items per second: 9.99
SLIMElasticNetRecommender: Processed 29922 (78.5%) in 50.01 min. Items per second: 9.97
SLIMElasticNetRecommender: Processed 32802 (86.0%) in 55.01 min. Items per second: 9.94
SLIMElasticNetRecommender: Processed

[I 2024-12-02 19:33:39,620] Trial 82 finished with value: 0.06024040517550655 and parameters: {'topK': 1496, 'l1_ratio': 0.19308065390530083, 'alpha': 0.00015362174543968933, 'positive_only': True}. Best is trial 56 with value: 0.060260283792308925.


SLIMElasticNetRecommender: Processed 2885 ( 7.6%) in 5.00 min. Items per second: 9.61
SLIMElasticNetRecommender: Processed 5626 (14.8%) in 10.00 min. Items per second: 9.37
SLIMElasticNetRecommender: Processed 8492 (22.3%) in 15.00 min. Items per second: 9.43
SLIMElasticNetRecommender: Processed 11360 (29.8%) in 20.00 min. Items per second: 9.46
SLIMElasticNetRecommender: Processed 14260 (37.4%) in 25.00 min. Items per second: 9.50
SLIMElasticNetRecommender: Processed 17413 (45.7%) in 30.01 min. Items per second: 9.67
SLIMElasticNetRecommender: Processed 20361 (53.4%) in 35.01 min. Items per second: 9.69
SLIMElasticNetRecommender: Processed 23106 (60.6%) in 40.01 min. Items per second: 9.63
SLIMElasticNetRecommender: Processed 26320 (69.0%) in 45.01 min. Items per second: 9.75
SLIMElasticNetRecommender: Processed 29586 (77.6%) in 50.01 min. Items per second: 9.86
SLIMElasticNetRecommender: Processed 32816 (86.1%) in 55.01 min. Items per second: 9.94
SLIMElasticNetRecommender: Processed

[I 2024-12-02 20:38:24,594] Trial 83 finished with value: 0.06019732127527993 and parameters: {'topK': 1476, 'l1_ratio': 0.19000860357479138, 'alpha': 0.00014442306978496852, 'positive_only': True}. Best is trial 56 with value: 0.060260283792308925.


...

[I 2024-12-02 21:45:33,045] Trial 84 finished with value: 0.06010574875640027 and parameters: {'topK': 1054, 'l1_ratio': 0.1380845412037027, 'alpha': 0.00015265859550523203, 'positive_only': True}. Best is trial 56 with value: 0.060260283792308925.


SLIMElasticNetRecommender: Processed 3104 ( 8.1%) in 5.00 min. Items per second: 10.34
SLIMElasticNetRecommender: Processed 6210 (16.3%) in 10.00 min. Items per second: 10.35
SLIMElasticNetRecommender: Processed 8895 (23.3%) in 15.00 min. Items per second: 9.88
SLIMElasticNetRecommender: Processed 11459 (30.1%) in 20.00 min. Items per second: 9.55
SLIMElasticNetRecommender: Processed 14224 (37.3%) in 25.00 min. Items per second: 9.48
SLIMElasticNetRecommender: Processed 17112 (44.9%) in 30.01 min. Items per second: 9.50
SLIMElasticNetRecommender: Processed 20039 (52.6%) in 35.01 min. Items per second: 9.54
SLIMElasticNetRecommender: Processed 23279 (61.1%) in 40.01 min. Items per second: 9.70
SLIMElasticNetRecommender: Processed 26730 (70.1%) in 45.01 min. Items per second: 9.90
SLIMElasticNetRecommender: Processed 29911 (78.5%) in 50.01 min. Items per second: 9.97
SLIMElasticNetRecommender: Processed 32726 (85.8%) in 55.01 min. Items per second: 9.92
SLIMElasticNetRecommender: Process

[I 2024-12-02 22:50:33,262] Trial 85 finished with value: 0.06010406870056939 and parameters: {'topK': 1406, 'l1_ratio': 0.12058138339127224, 'alpha': 0.00016456752786783398, 'positive_only': True}. Best is trial 56 with value: 0.060260283792308925.


SLIMElasticNetRecommender: Processed 2742 ( 7.2%) in 5.00 min. Items per second: 9.14
SLIMElasticNetRecommender: Processed 5417 (14.2%) in 10.00 min. Items per second: 9.03
SLIMElasticNetRecommender: Processed 7883 (20.7%) in 15.00 min. Items per second: 8.76
SLIMElasticNetRecommender: Processed 10369 (27.2%) in 20.00 min. Items per second: 8.64
SLIMElasticNetRecommender: Processed 12998 (34.1%) in 25.00 min. Items per second: 8.66
SLIMElasticNetRecommender: Processed 15514 (40.7%) in 30.01 min. Items per second: 8.62
SLIMElasticNetRecommender: Processed 18192 (47.7%) in 35.01 min. Items per second: 8.66
SLIMElasticNetRecommender: Processed 20987 (55.1%) in 40.01 min. Items per second: 8.74
SLIMElasticNetRecommender: Processed 23471 (61.6%) in 45.01 min. Items per second: 8.69
SLIMElasticNetRecommender: Processed 26090 (68.4%) in 50.01 min. Items per second: 8.69
SLIMElasticNetRecommender: Processed 28590 (75.0%) in 55.01 min. Items per second: 8.66
SLIMElasticNetRecommender: Processed

[I 2024-12-03 00:03:59,338] Trial 86 finished with value: 0.05976723701607654 and parameters: {'topK': 1434, 'l1_ratio': 0.2643259762185103, 'alpha': 0.0001294649009400823, 'positive_only': False}. Best is trial 56 with value: 0.060260283792308925.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/best_params_{config["model"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    recommender_instance = SLIMElasticNetRecommender(URM_train + URM_validation)
    recommender_instance.fit(**best_params)

SLIMElasticNetRecommender: Processed 1933 ( 5.1%) in 5.00 min. Items per second: 6.44
SLIMElasticNetRecommender: Processed 4032 (10.6%) in 10.00 min. Items per second: 6.72
SLIMElasticNetRecommender: Processed 6102 (16.0%) in 15.00 min. Items per second: 6.78
SLIMElasticNetRecommender: Processed 8160 (21.4%) in 20.00 min. Items per second: 6.80
SLIMElasticNetRecommender: Processed 10501 (27.5%) in 25.00 min. Items per second: 7.00
SLIMElasticNetRecommender: Processed 12639 (33.2%) in 30.00 min. Items per second: 7.02
SLIMElasticNetRecommender: Processed 14763 (38.7%) in 35.00 min. Items per second: 7.03
SLIMElasticNetRecommender: Processed 16842 (44.2%) in 40.01 min. Items per second: 7.02
SLIMElasticNetRecommender: Processed 18798 (49.3%) in 45.01 min. Items per second: 6.96
SLIMElasticNetRecommender: Processed 20912 (54.9%) in 50.01 min. Items per second: 6.97
SLIMElasticNetRecommender: Processed 22933 (60.2%) in 55.01 min. Items per second: 6.95
SLIMElasticNetRecommender: Processed 

# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}.csv')

Submission file saved as /kaggle/working/submission_SLIMElasticNet.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/best_params_{config["model"]}.json', 
            f'{config["model"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/'\
        f'best_params_{config["model"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}.json'
    )

File 'TrainedModels/WithoutKFCV/SLIM/SLIMElasticNetRecommender/best_params_SLIMElasticNet.json' updated successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/history_{config["model"]}.db',
        f'Tuning {config["model"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithoutKFCV/SLIM/SLIMElasticNetRecommender/history_SLIMElasticNet.db' updated successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Submission/submission_{config["model"]}.csv', 
                f'New {config["model"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithoutKFCV/SLIM/SLIMElasticNetRecommender/Submission/submission_SLIMElasticNet.csv' updated successfully.
